# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""31462474281028…","""13770120213171…","""../../../../da…",1.2927e19,"""65287343313807…","""13871792732777…",false
"""53239797866065…","""13697966901779…","""../../../../da…",1.4856e19,"""65287343313807…","""34216544204907…",true
"""36897406992273…","""84117707542633…","""../../../../da…",5.6458e17,"""16537993835248…","""13871792732777…",false
"""53239797866065…","""17075732572912…","""../../../../da…",4.0723e18,"""14559841333218…","""13871792732777…",false
"""31462474281028…","""17538153634940…","""../../../../da…",1.4677e19,"""14559841333218…","""55511853250136…",true
"""36897406992273…","""17924056036335…","""../../../../da…",1.6330e19,"""40362579710948…",null,false
"""36897406992273…","""13959188296919…","""../../../../da…",8.7675e18,"""14559841333218…","""55511853250136…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""17642868909740…","""17532382345781…","""../../../../da…",1.2927e19,"""FirstMortgage3…","""62036753254679…","""Rejected"""
"""17555750693462…","""13508434696110…","""../../../../da…",1.4856e19,"""FirstMortgage3…","""14350687268024…","""Accepted"""
"""13971003329687…","""29328270740493…","""../../../../da…",5.6458e17,"""MoneyMarketSav…","""62036753254679…","""Rejected"""
"""17555750693462…","""11320307242879…","""../../../../da…",4.0723e18,"""BasicChecking""","""62036753254679…","""Rejected"""
"""17642868909740…","""16320942495057…","""../../../../da…",1.4677e19,"""BasicChecking""","""25105167096797…","""Accepted"""
"""13971003329687…","""12590255965401…","""../../../../da…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""13971003329687…","""12945147571147…","""../../../../da…",8.7675e18,"""BasicChecking""","""25105167096797…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""73956017921264…","""22273356178601…","""../../../../da…",1.2927e19,"""FirstMortgage3…","""15201400732449…","""Rejected"""
"""35650349235897…","""40879582779695…","""../../../../da…",1.4856e19,"""FirstMortgage3…","""15472712937097…","""Accepted"""
"""11662947903304…","""16369213593397…","""../../../../da…",5.6458e17,"""MoneyMarketSav…","""15201400732449…","""Rejected"""
"""35650349235897…","""89358688420007…","""../../../../da…",4.0723e18,"""BasicChecking""","""15201400732449…","""Rejected"""
"""73956017921264…","""63545614052895…","""../../../../da…",1.4677e19,"""BasicChecking""","""93470014423172…","""Accepted"""
"""11662947903304…","""59508258350071…","""../../../../da…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""11662947903304…","""10802362063188…","""../../../../da…",8.7675e18,"""BasicChecking""","""93470014423172…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""57544635171770…","""18303821820633…",1.2927e19,"""FirstMortgage3…","""10397386206268…","""Rejected"""
"""81870452912648…","""11806006577433…",1.4856e19,"""FirstMortgage3…","""38488793734782…","""Accepted"""
"""71254569613952…","""52624926087246…",5.6458e17,"""MoneyMarketSav…","""10397386206268…","""Rejected"""
"""81870452912648…","""84760915695788…",4.0723e18,"""BasicChecking""","""10397386206268…","""Rejected"""
"""57544635171770…","""15781502199285…",1.4677e19,"""BasicChecking""","""10663716791257…","""Accepted"""
"""71254569613952…","""81045973308296…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""71254569613952…","""45107008452873…",8.7675e18,"""BasicChecking""","""10663716791257…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'